In [12]:
import pandas as pd 
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

ModuleNotFoundError: No module named 'nltk'

### Notes
If system package installation fails, use pip to install NLTK in your environment:
`pip install nltk`

### Environment Setup
A Python virtual environment named `nlpEnv` has been created. Activate it before running the notebook:
- Linux/macOS: `source nlpEnv/bin/activate`
- Windows (PowerShell): `nlpEnv\Scripts\Activate.ps1`
Then install NLTK and required resources:

In [ ]:
# Install nltk in the active environment (run once)
import sys, subprocess
subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'nltk'])

import nltk
nltk.download('stopwords')
nltk.download('punkt')

### Install/Verify NLTK
NLTK should be installed in the active environment. If not, run the following cell to install via pip.
If you prefer system packages on Debian/Ubuntu: `sudo apt-get update && sudo apt-get install -y python3-nltk` (then restart kernel).

In [ ]:
# Install NLTK in current Python environment if missing
try:
    import nltk
    print('NLTK version:', nltk.__version__)
except Exception:
    import sys, subprocess
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'nltk'])
    import nltk
    print('Installed NLTK version:', nltk.__version__)

In [ ]:
# Ensure required NLTK resources are available
import nltk
nltk.download('stopwords')
nltk.download('punkt')